# Surgery Process
### Continued Analysis

The diagnostic imaging (DI) team has provided you with data about patients they have seen in their clinic.   They have sent an extract of all patients seen for abdominal scans during the same time period as the OR Booking data.  This exercise will be to explore the patient journey through the DI clinic for the patients who had Appendectomy’s & Choles.

In [86]:
# Import necessary libraries
import pandas as pd

* Combining the DI Requisition csv files into a single table.

In [87]:
di1 = pd.read_excel('data/DI - Visits 1.3.xlsx')
di2 = pd.read_excel('data/DI - Visits 2.3.xlsx')
di3 = pd.read_excel('data/DI - Visits 3.3.xlsx')

print('Check if they have same columns')
print('DI - Visits 1.3 columns:\n', di1.columns)
print('DI - Visits 2.3 columns:\n', di2.columns)
print('DI - Visits 3.3 columns:\n', di3.columns)

print('\nCheck shapes:')
print('DI - Visits 1.3 shape:\n', di1.shape)
print('DI - Visits 2.3 shape:\n', di2.shape)
print('DI - Visits 3.3 shape:\n', di3.shape)

Check if they have same columns
DI - Visits 1.3 columns:
 Index(['HCID', 'Pt Age', 'DI Req - Time', 'Requesting Physician',
       'Req Type - Abdominal', 'DI - Pt in Suite'],
      dtype='object')
DI - Visits 2.3 columns:
 Index(['HCID', 'Pt Age', 'DI Req - Time', 'Requesting Physician',
       'Req Type - Abdominal', 'DI - Pt in Suite'],
      dtype='object')
DI - Visits 3.3 columns:
 Index(['HCID', 'Pt Age', 'DI Req - Time', 'Requesting Physician',
       'Req Type - Abdominal', 'DI - Pt in Suite'],
      dtype='object')

Check shapes:
DI - Visits 1.3 shape:
 (820, 6)
DI - Visits 2.3 shape:
 (820, 6)
DI - Visits 3.3 shape:
 (817, 6)


In [88]:
# Concat 3 DI dataframes to one table
di_records = pd.concat([di1, di2, di3], axis=0)
display(di_records.head())
print(di_records.shape)
di_records.info()

# Check null
print('\nCheck null values:')
display(di_records.isnull().sum())

# Check duplicates
print('\nCheck duplicate records:')
di_records[(di_records.duplicated(subset=['HCID']))]

,HCID,Pt Age,DI Req - Time,Requesting Physician,Req Type - Abdominal,DI - Pt in Suite
0,2203144,70.0,2019-01-21 15:27:50.013,Dr. S,ABD,2019-01-21 16:42:31.173
1,2461112,67.0,2019-01-21 15:27:50.013,Dr. S,ABD,2019-01-21 18:43:27.644
2,2233815,58.0,2019-01-21 15:43:24.757,Dr. S,ABD,2019-01-21 15:55:06.378
3,1043375,31.0,2019-01-22 01:19:36.383,Dr. S,ABD,2019-01-22 02:53:00.812
4,2933761,60.0,2019-01-22 02:20:09.396,Dr. O,ABD,2019-01-22 03:41:22.028


(2457, 6)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2457 entries, 0 to 816
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   HCID                  2457 non-null   int64         
 1   Pt Age                2412 non-null   float64       
 2   DI Req - Time         2457 non-null   datetime64[ns]
 3   Requesting Physician  2457 non-null   object        
 4   Req Type - Abdominal  2457 non-null   object        
 5   DI - Pt in Suite      2457 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1), object(2)
memory usage: 134.4+ KB

Check null values:


HCID                     0
Pt Age                  45
DI Req - Time            0
Requesting Physician     0
Req Type - Abdominal     0
DI - Pt in Suite         0
dtype: int64


Check duplicate records:


,HCID,Pt Age,DI Req - Time,Requesting Physician,Req Type - Abdominal,DI - Pt in Suite
108,2155841,87.0,2019-12-09 21:31:56.911,Dr. O,ABD,2019-12-10 00:29:10.009
717,2286275,73.0,2020-04-01 09:22:07.521,Dr. S,ABD,2020-04-01 12:01:53.279


In [89]:
# Check duplicate records
display(di_records[(di_records['HCID']==2155841)])

display(di_records[(di_records['HCID']==2286275)])

,HCID,Pt Age,DI Req - Time,Requesting Physician,Req Type - Abdominal,DI - Pt in Suite
806,2155841,58.0,2019-01-22 02:20:09.396,Dr. D,ABD,2019-11-17 11:29:52.714
108,2155841,87.0,2019-12-09 21:31:56.911,Dr. O,ABD,2019-12-10 00:29:10.009


,HCID,Pt Age,DI Req - Time,Requesting Physician,Req Type - Abdominal,DI - Pt in Suite
652,2286275,44.0,2019-10-23 11:00:28.223,Dr. P,ABD,2019-10-23 14:55:54.745
717,2286275,73.0,2020-04-01 09:22:07.521,Dr. S,ABD,2020-04-01 12:01:53.279


**NOTE:**
We will take note this two HCIDs that have duplicate records and asked the SME if we can drop these records.

* The DI department does not have information on why patients were seen there. Using the OR Booking data to determine which records from DI were from patients who were in the care pathway for an Appendectomy or Cholecystectomy. 

In [90]:
or_booking = pd.read_csv('data/OR Booking.csv')
# Trim space on HCID column of OR Booking
or_booking.rename(columns={'HCID ': 'HCID'}, inplace=True)

di_orbooking = di_records.merge(or_booking, how='left', on='HCID')
patients = di_orbooking[(di_orbooking['Proc Descr Mod'].str.contains('Appendectomy')) |
                    (di_orbooking['Proc Descr Mod'].str.contains('Cholecystectomy'))]
print('Total number of patients in Diagnostic Imaging:', di_orbooking.shape[0])
print('Total number of patients in Diagnostic Imaging who were for Appendectomy or Cholecystectomy:', patients.shape[0])

Total number of patients in Diagnostic Imaging: 2457
Total number of patients in Diagnostic Imaging who were for Appendectomy or Cholecystectomy: 731


Use DI records on the left to accurately capture the number of cases related to Appendectomy or Cholecystectomy. These records will serve as the foundational list items for evaluating and ensuring the effectiveness of our care pathway.

In [91]:
or_di_data = or_booking.merge(di_records, how='left', on=['HCID', 'Pt Age'])
print(or_di_data.info)

# Convert OR Booking Req DT/Tm to date time
or_di_data['OR Booking Req DT/Tm'] = pd.to_datetime(or_di_data['OR Booking Req DT/Tm'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

<bound method DataFrame.info of          HCID  Pt Age               Proc Descr Mod Req Proc Tm Pt Priority  \
0     1805294    40.0     Laparoscopy Appendectomy        0060       E-06H   
1     2233815    58.0  Laparoscopy Cholecystectomy        0060       E-24H   
2     1043375    31.0     Laparoscopy Appendectomy        0060       E-06H   
3     1203917    45.0  Laparoscopy Cholecystectomy        0060       E-24H   
4     2616633    46.0     Laparoscopy Appendectomy                   E-06H   
...       ...     ...                          ...         ...         ...   
1224  1547397    25.0     Laparoscopy Appendectomy        0060       E-06H   
1225  2250377    54.0  Laparoscopy Cholecystectomy        0060       E-24H   
1226  2803120    18.0  Laparoscopy Cholecystectomy        0060       U-36H   
1227  1602995    25.0     Laparoscopy Appendectomy        0060       E-06H   
1228  2939119    41.0     Laparoscopy Appendectomy        0060       E-06H   

     OR Booking Req DT/Tm Pt Lo

* Get the percentage of Appendectomy patients going through DI on their pathway to the OR.

In [92]:
# Get Appendectomy patients
appende_patients = or_di_data[(or_di_data['Proc Descr Mod'].str.contains('Appendectomy'))]

# Get Appendectomy patients that go through DI
appende_di_patients = appende_patients[(appende_patients['DI Req - Time'].isna()== False)]

# Get percentage of Appendectomy patients that gone through DI
appendectomy_di_percent = 100 * (appende_di_patients.shape[0] / appende_patients.shape[0])

print(f'Total Appendectomy patients: {appende_patients.shape[0]}')
print(f'Total Appendectomy patients that gone through DI: {appende_di_patients.shape[0]}')
print(f'Percentage of Appendectomy patients that gone through DI: {appendectomy_di_percent:.2f}%')

Total Appendectomy patients: 578
Total Appendectomy patients that gone through DI: 194
Percentage of Appendectomy patients that gone through DI: 33.56%


* Get the percentage of Cholecystectomy patients going through DI on their pathway to the OR.

In [93]:
# Get Cholecystectomy patients
chole_patients = or_di_data[(or_di_data['Proc Descr Mod'].str.contains('Cholecystectomy'))]

# Get Cholecystectomy patients that go through DI
chole_di_patients = chole_patients[(chole_patients['DI Req - Time'].isna()== False)]

# Get percentage of Cholecystectomy patients that gone through DI
cholecystectomy_di_percent = 100 * (chole_di_patients.shape[0] / chole_patients.shape[0])

print(f'Total Cholecystectomy patients: {chole_patients.shape[0]}')
print(f'Total Cholecystectomy patients that gone through DI: {chole_di_patients.shape[0]}')
print(f'Percentage of Cholecystectomy patients that gone through DI: {cholecystectomy_di_percent:.2f}%')

Total Cholecystectomy patients: 651
Total Cholecystectomy patients that gone through DI: 537
Percentage of Cholecystectomy patients that gone through DI: 82.49%


### Is there any group more likely to go through DI prior to the OR?

**Answer:**

Based on the data provided, it is evident that the Adult, Middle Aged, and Aged groups are more likely to undergo a Diagnostic Image procedure before an operation. This trend makes sense as individuals aged 25 and above tend to engage in more activities compared to young adults and children. The statistics clearly support the rationale behind this pattern.

In [94]:
# Make age groupings
def get_age_group(age):
    if age <= 1:
        return 'Infant'
    elif age > 1 and age <= 3:
        return 'Toddler'
    elif age > 3 and age <= 12:
        return 'Child'
    elif age > 12 and age <= 19:
        return 'Teen'
    elif age > 19 and age <= 24:
        return 'Young Adult'
    elif age > 24 and age <= 44:
        return 'Adult'
    elif age > 44 and age <= 64:
        return 'Middle Aged'
    else:
        return 'Aged'

or_di_data['Age Group'] = or_di_data['Pt Age'].apply(get_age_group)
or_di_data[['HCID', 'Proc Descr Mod', 'Age Group']]\
            .where(or_di_data['DI Req - Time'].isna()== False)\
            .groupby(['Proc Descr Mod', 'Age Group']).count()

HCID
Proc Descr Mod              Age Group        
Laparoscopy Appendectomy    Adult          50
                            Aged           39
                            Middle Aged    80
                            Teen           11
                            Young Adult    14
Laparoscopy Cholecystectomy Adult         195
                            Aged          143
                            Middle Aged   165
                            Teen           11
                            Young Adult    23

### Are there particular days of the week where wait times to be seen by DI are significantly higher or lower than the overall average for Appendectomy or Cholecystectomy patients?

**Answer:**

From the data below, it has been observed that there is significant high waiting time (10091.28 seconds or 2.8 hours) during Wednesday while there is a minimal wait time (10.5 minutes) during Sunday.

In [95]:
# Compute the wait time in DI
or_di_data['DIWaitTime'] = (or_di_data['DI - Pt in Suite'] - or_di_data['DI Req - Time']).dt.total_seconds()

# Filter Appendectomy and Cholecystectomy patients with valid wait time >= 0
di_wait_records = or_di_data[((or_di_data['Proc Descr Mod'].str.contains('Appendectomy')) |
                            (or_di_data['Proc Descr Mod'].str.contains('Cholecystectomy'))) &
                            (or_di_data['DIWaitTime'] >= 0)]
print('Total valid DI wait time di_wait_records:', di_wait_records.shape[0])

average_wait = di_wait_records['DIWaitTime'].mean()
print(f'Average DI wait time: {average_wait:.2f} seconds')

p90 = di_wait_records['DIWaitTime'].quantile(0.9)
p10 = di_wait_records['DIWaitTime'].quantile(0.1)
print(f'90% above average Wait Time: {p90:.2f} seconds')
print(f'10% above average Wait Time: {p10:.2f} seconds')

# Get significantly higher or lower than the overall average wait time
high_wait = di_wait_records[(di_wait_records['DIWaitTime'] >= p90)]
print('\nHigh Wait Time Rank- Day of the Week')
display(pd.DataFrame(high_wait['DI Req - Time'].dt.day_name()).value_counts())

low_wait = di_wait_records[(di_wait_records['DIWaitTime'] <= p10)]
print('Low Wait Time Rank - Day of the Week')
display(pd.DataFrame(low_wait['DI Req - Time'].dt.day_name()).value_counts())

Total valid DI wait time di_wait_records: 731
Average DI wait time: 4850.43 seconds
90% above average Wait Time: 10091.28 seconds
10% above average Wait Time: 630.31 seconds

High Wait Time Rank- Day of the Week


DI Req - Time
Wednesday        16
Monday           12
Thursday         12
Saturday         11
Sunday           11
Friday            7
Tuesday           5
dtype: int64

Low Wait Time Rank - Day of the Week


DI Req - Time
Sunday           15
Thursday         11
Wednesday        11
Friday           10
Saturday         10
Monday            9
Tuesday           8
dtype: int64

### Are there particular times of the day where wait times to be seen by DI are significantly higher or lower than the overall average for Appendectomy or Cholecystectomy patients?

**Answer:**

From the data, it has been observed that there is significant high waiting time (10091.28 seconds or 2.8 hours) during 6 and 9 in the morning but there will be a minimal wait time (10.5 minutes) during 10 to 11 in the morning.

In [96]:
# Get significantly higher or lower than the overall average wait time
print('\nHigh Wait Time Rank- Time of the Day')
display(pd.DataFrame(high_wait['DI Req - Time'].dt.hour).value_counts())

print('Low Wait Time Rank - Time of the Day')
display(pd.DataFrame(low_wait['DI Req - Time'].dt.hour).value_counts())


High Wait Time Rank- Time of the Day


DI Req - Time
6                7
9                6
4                5
5                5
7                5
8                5
11               5
13               5
10               4
20               4
16               4
3                3
17               3
15               2
23               2
12               2
1                2
2                2
14               1
22               1
0                1
dtype: int64

Low Wait Time Rank - Time of the Day


DI Req - Time
10               7
11               7
16               6
7                6
15               6
9                6
17               5
6                4
1                4
18               3
8                3
5                3
19               2
20               2
22               2
23               2
21               1
0                1
14               1
12               1
4                1
13               1
dtype: int64

Sometimes data can be used to find quality improvement opportunities.  The idea has been considered that the time difference between the patient being seen by DI (DI - Pt in Suite) and the time a patient is booked into the OR (OR Booking Req DT/Tm) is reflective of a doctor’s ability to manage their workload.  

### Is there a doctor who is significantly better than their peers in this area for one or both of the procedures?

In [97]:
or_di_data['DI To OR Interval (s)'] = (or_di_data['OR Booking Req DT/Tm'] - or_di_data['DI - Pt in Suite']).dt.total_seconds()

# Filter Appendectomy and Cholecystectomy patients only and with valid interval time
or_to_di_records = or_di_data[((or_di_data['Proc Descr Mod'].str.contains('Appendectomy')) |
                    (or_di_data['Proc Descr Mod'].str.contains('Cholecystectomy'))) &
                    (or_di_data['DI To OR Interval (s)'].isnull() == False) &
                    (or_di_data['DI To OR Interval (s)'] >= 0.0)]
top5_doctor = or_to_di_records.nsmallest(n=5, columns='DI To OR Interval (s)')
print('Top 5 Efficient Physicians with minimal DI to OR internal time:')
display(top5_doctor[['Requesting Physician', 'Proc Descr Mod', 'DI To OR Interval (s)']])

Top 5 Efficient Physicians with minimal DI to OR internal time:


,Requesting Physician,Proc Descr Mod,DI To OR Interval (s)
214,Dr. S,Laparoscopy Cholecystectomy,28.701
968,Dr. J,Laparoscopy Cholecystectomy,50.241
164,Dr. S,Laparoscopy Appendectomy,51.931
55,Dr. J,Laparoscopy Appendectomy,78.132
564,Dr. S,Laparoscopy Appendectomy,103.411
